Состав группы: Анна Смекалова (БКЛ213), Александра Сидоркина (БКЛ211), Александра Черняева (БКЛ212)

#### Импорты

In [139]:
import pandas as pd
import numpy as np
import nltk
from collections import Counter
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score

#### Подготовка данных

In [22]:
df = pd.read_csv('train.csv', delimiter=',')

In [23]:
data = df['text']
answers = df['answer']

Используем функцию **preprocess_text**, которая позволит привести текст каждого ревью в необходимый вид для дальнейшей векторизации, и функцию **get_pos**, которая помогает определять часть речи для более эффективной лемматизации через WordNetLemmatizer и метод lemmatize() 

In [34]:
def get_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [35]:
def preprocess_text(text):
    wnl = WordNetLemmatizer() #лемматизатор
    stop_words = set(stopwords.words("english")) #стоп-слова
    # токенизация текста отзыва
    tokens = word_tokenize(text) 
    # приведение всех символов к нижнему регистру + удаление пунктуации
    tokens = [word.lower() for word in tokens if word.isalpha()] 
    # удаление стоп-слов
    tokens = [word for word in tokens if not word in stop_words]
    # лемматизиция
    tokens = [wnl.lemmatize(word, get_pos(word)) for word in tokens]
    
    return " ".join(tokens) 

In [38]:
data = data.apply(preprocess_text)

Преобразуем тексты в вектора при помощи TF-IDF:

In [100]:
tfidf = TfidfVectorizer(
    max_features=10000

)
x_data = tfidf.fit_transform(data)
feature_names = tfidf.get_feature_names_out()

Далее разделим данные на обучающую и тестовую выборку, ограничив количество текстов, на которых мы будем обучать и тестировать наши модели в поиске самой эффективной из них:

In [101]:
X_train, X_test, y_train, y_test = train_test_split(x_data[:10000], answers[:10000], test_size=0.2, random_state=0)

In [102]:
print(X_train.shape)
print(X_test.shape)

(8000, 10000)
(2000, 10000)


#### Модель 1: логистическая регрессия

In [140]:
Counter(y_train)

Counter({0: 4009, 1: 3991})

Классы в обучающей выборке оказались представлены не совсем в равном соотношении, поэтому устанавливаем class_weight='balanced':

In [215]:
log_reg = LogisticRegression(class_weight='balanced')
log_reg.fit(X_train, y_train)

LogisticRegression(class_weight='balanced')

#### Модель 2: дерево решений

In [216]:
dt = DecisionTreeClassifier(max_depth=20, class_weight='balanced')
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=20)

#### Модель 3: случайный лес

In [217]:
rf = RandomForestClassifier(n_estimators=200, class_weight='balanced')
rf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', n_estimators=200)

#### Выводы

In [218]:
models = [log_reg, dt, rf]
model_names = ['Logistic Regression', 'Decision Tree Classifier', 'Random Forest Classifier']

In [219]:
for model, model_name in zip(models, model_names):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(f'Accuracy of {model_name}: {accuracy}')
    print(f'F1 of {model_name}: {f1}\n')

Accuracy of Logistic Regression: 0.846
F1 of Logistic Regression: 0.8467661691542288

Accuracy of Decision Tree Classifier: 0.706
F1 of Decision Tree Classifier: 0.7106299212598425

Accuracy of Random Forest Classifier: 0.8245
F1 of Random Forest Classifier: 0.8220983274201723



Лучше всего себя показала логистическая регрессия. Попробуем обучить логистическую модель уже не на части, а на всех данных

In [220]:
Counter(answers)

Counter({0: 12500, 1: 12500})

In [221]:
best_model = LogisticRegression()
best_model.fit(x_data, answers)

LogisticRegression()

In [222]:
cross_val_score(best_model, X=x_data, y=answers, cv=10)

array([0.8864, 0.8756, 0.88  , 0.8784, 0.8744, 0.8816, 0.8852, 0.8952,
       0.8772, 0.8864])

#### Результаты для test.csv

Обрабатываем тексты в test.csv по тому же алгоритму, что обрабатывали тексты в train.csv:

In [56]:
df_test = pd.read_csv('test.csv', delimiter=',')
data_test = df_test['text']
data_test = data_test.apply(preprocess_text)

In [223]:
X_data_test = tfidf.fit_transform(data_test)

In [224]:
print(X_data_test.shape)

(25000, 10000)


Предсказываем значения для каждого из текстов в text.csv с помощью модели best_model, обученной на всём наборе данных

In [225]:
y_pred_test = best_model.predict(X_data_test)

Сохраняем ответы в result.csv

In [226]:
result = df_test[["id"]].copy()
result["answer"] = y_pred_test 
result.to_csv("result.csv", index=False)